In [60]:
import pandas as pd
import numpy as np

In [15]:
link1 = "./BD/df_tID.csv"
link2 = "./BD/df_nID.csv"
link3 = "./BD/df_tmdb_clean.csv"
link4 = "./BD/df_t_principals_clean.csv"

df_tID = pd.read_csv(link1)
df_nID = pd.read_csv(link2)
df_tmdb_clean = pd.read_csv(link3)
df_t_principals = pd.read_csv(link4)

Ajout d'une colonne "Liste Acteurs" et colonne "Réalisatuers" dans df_tmdb-clean

In [31]:
#On filtre les categories du df_t_principals sur des nouveaux df pour un futur merge:
df_t_principals_act = df_t_principals[(df_t_principals['category'] == 'actor') | (df_t_principals['category'] == 'actress')]
df_t_principals_real = df_t_principals[(df_t_principals['category'] == 'director')]

#On drop la colonne maintenant inutile 'category' est on groupby sur les tconst pour le merge:
df_liste_act = df_t_principals_act.drop('category', axis = 1).groupby('tconst').agg(list)
df_liste_real = df_t_principals_act.drop('category', axis = 1).groupby('tconst').agg(list)

#Premier merge pour la colonne "Liste Acteurs":
df_recherche = pd.merge(df_tmdb_clean, df_liste_act, how = 'left', left_on = 'ID', right_on = 'tconst').rename(columns = {'nconst' : 'Liste acteurs'})

#2e pour l'ajout de la colonne "Réalisateurs":
df_recherche = pd.merge(df_recherche, df_liste_real, how = 'left', left_on = 'ID', right_on = 'tconst').rename(columns = {'nconst' : 'Réalisateurs'})

#Il reste des lignes NaN et ce sont les même, on les garde en NaN et ainsi elle ne pourront pas être mentionnées par le système de recherche

In [57]:
df_recherche

,ID,Titre,Date de sortie,Durée (minutes),Genres,Langue Originale,Note moyenne,Nb de votes,Score de popularité,Résumé,Lien du poster,Budget,Titres possibles,Décénnie,Liste acteurs,Réalisateurs
0,tt0054823,Doctor Blood's Coffin,1961-01-01,92,['Horror'],en,5.00,23,1.452,After being thrown out of medical school for e...,/7GkBT8cKPl1GhBuUnktJpjF1uV.jpg,0,"['Le cadavre qui tue', 'O Esquife do Morto-Viv...",1960,"[nm0601476, nm0183764, nm0402842, nm0912915, n...","[nm0601476, nm0183764, nm0402842, nm0912915, n..."
1,tt0054886,Follow That Man,1961-01-01,84,['Comedy'],en,5.80,1,0.841,A farcical comedy about a con-man and a girl r...,/xxcZ014zEsJm7WW8k5urHEhr783.jpg,0,['Follow That Man'],1960,"[nm0152261, nm0011625, nm0545294, nm0372214, n...","[nm0152261, nm0011625, nm0545294, nm0372214, n..."
2,tt0054807,The Devil's Messenger,1961-01-01,72,['Horror'],en,4.30,13,1.130,In this feature version of the Swedish TV seri...,/yBfnRV6YNrZM33o1dwdjdQSoZGl.jpg,0,"['La mensajera del diablo', 'La messaggera del...",1960,"[nm0001033, nm0434377, nm0385827]","[nm0001033, nm0434377, nm0385827]"
3,tt0054996,House of Mystery,1961-01-01,56,"['Mystery', 'Horror']",en,6.60,4,0.961,A young couple go to check out a house they're...,/9YVj1dwT4TXM59rytnAoCeySjWO.jpg,0,"['Das Landhaus des Dr. Lemming', 'House of Mys...",1960,NaN,NaN
4,tt0055247,Offbeat,1961-01-01,71,['Crime'],en,6.25,8,1.754,When Scotland Yard finds themselves up against...,/sO2oyJDMUCdQsuJIhcg5kM9aRh1.jpg,0,"['Polisspionen', 'In den Fängen des FBI', 'O t...",1960,"[nm0843213, nm0955195, nm0576851, nm0206060, n...","[nm0843213, nm0955195, nm0576851, nm0206060, n..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79891,tt17527468,Bottoms,2023-08-25,92,['Comedy'],en,5.80,1,6.615,Two unpopular queer girls in their senior year...,/yiiCaamwKb5p2kD95jsjdGJRmTn.jpg,0,"['ボトムス ～最底で最強？な私たち～', 'Na dnie', 'Passivonas',...",2020,"[nm6976735, nm8731249, nm9342097, nm10023988, ...","[nm6976735, nm8731249, nm9342097, nm10023988, ..."
79892,tt26448981,"Orlando, ma biographie politique",2023-10-11,98,['Documentary'],fr,5.80,1,2.411,Virginia Woolf’s “Orlando” tells the story of ...,/dsEsYLBgo9pIumB3kgToDWuOg7x.jpg,0,"['千面歐蘭朵', 'Orlando, Minha Biografia Política',...",2020,"[nm14598153, nm14598151, nm14543939, nm0067476...","[nm14598153, nm14598151, nm14543939, nm0067476..."
79893,tt26347358,Notre corps,2023-10-11,168,['Documentary'],fr,5.80,1,2.426,"In a Parisian public hospital, Claire Simon qu...",/oamkG0bRZbsVS27mOrMTbz6UYNY.jpg,980000,"['Nossos Corpos', 'Notre corps', 'Nasze ciało'...",2020,NaN,NaN
79894,tt15683734,It Lives Inside,2023-11-03,99,"['Drama', 'Horror', 'Thriller']",en,5.80,1,3.508,"Sam is desperate to fit in at school, rejectin...",/dpgI82d8CrCDqDdBQeDUeFOIzI4.jpg,0,"['It Lives Inside', 'Służka', ""Motzi Ta'Nshama...",2020,"[nm3514372, nm1263093, nm8860518, nm3340597, n...","[nm3514372, nm1263093, nm8860518, nm3340597, n..."


Maintenant On imagine que l'on demande à un utilisateur de rentrer des infos sur le film:

1. "Genre" en menu déroulant => choix = G
2. "Décénnie" en menu déroulant => choix = D
3. "Acteur" en barre recherche => choix = A
4. "Réalisateur" en barre recherche => choix = R


In [58]:
df_reponse = df_recherche.copy()

In [ ]:
df_reponse = df_recherche.copy()
if G != np.nan:
    df_reponse = df_reponse[df_reponse['Genres'].str.contains(G)]
if D != np.nan:
    df_reponse = df_reponse[df_reponse['Décénnie'] == D]
if A != np.nan:
    id_A = df_nID[df_nID['Nom'] == A]['n_ID'].values[0]
    df_reponse = df_reponse[df_reponse['Liste acteurs'].apply(lambda x: id_A in x)]


    


In [62]:
A = 'Tom Cruise'

In [ ]:
id_A = df_nID[df_nID['Nom'] == A]['n_ID'].values[0]

In [74]:
df_reponse = df_reponse[df_reponse['Liste acteurs'].str]

SyntaxError: invalid syntax (2698376475.py, line 1)

In [68]:
df_reponse.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79896 entries, 0 to 79895
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   79896 non-null  object 
 1   Titre                79896 non-null  object 
 2   Date de sortie       79896 non-null  object 
 3   Durée (minutes)      79896 non-null  int64  
 4   Genres               79896 non-null  object 
 5   Langue Originale     79896 non-null  object 
 6   Note moyenne         79896 non-null  float64
 7   Nb de votes          79896 non-null  int64  
 8   Score de popularité  79896 non-null  float64
 9   Résumé               79896 non-null  object 
 10  Lien du poster       79896 non-null  object 
 11  Budget               79896 non-null  int64  
 12  Titres possibles     79896 non-null  object 
 13  Décénnie             79896 non-null  int64  
 14  Liste acteurs        67869 non-null  object 
 15  Réalisateurs         67869 non-null 